# Importing Libraries

In [2]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install better_profanity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
%pip install torch transformers datasets ipywidgets flaml[blendsearch,ray] #It is important to run this before installing datasets, due to compatibility issues

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
#Importing all necessary packages and libraries

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import emoji
import string
import random
import time
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
import torch
from tqdm import tqdm
import spacy
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from better_profanity import profanity

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
nlp = spacy.blank("en")
%matplotlib inline

### Define Path

In [8]:
MODEL_PATH = "/content/drive/My Drive/ML4NLP_Assignment5/model/" #That's where we will store the model 
DATA_PATH = "/content/drive/My Drive/ML4NLP_Assignment5/data/" #That's where we will store the data

### Loading Dataset and Mounting Drive

In [9]:
# MOUNTING GOOGLE DRIVE

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [10]:
# FUNCTION FOR LOADING DATA

def load_data(data_path, file_name, label=False):
    data_path = data_path + file_name
    df = pd.read_csv(data_path, on_bad_lines="skip", sep = "\t")
    X = df["Tweet"]
    y = df["Intensity Score"]
    return X, y

In [12]:
# LOADING DATA

X_train_raw, y_train_raw = load_data(DATA_PATH, "EI-reg-En-anger-train.txt")   #raw train dataset
X_test_raw, y_test_raw = load_data(DATA_PATH, "2018-EI-reg-En-anger-test-gold.txt") #raw test dataset

### Data Cleaning & Tokenising

In [14]:
#Class for data cleaning and pre-processing 

class DataCleaning:
    def __init__(self):
        return
    # LOWERCASING
    def convert_lowercase(self, text):
        text = text.str.lower()
        return text
    # REMOVING HTML TAGS AND URLS
    def remove_html_url(self, text):
        re_html = re.compile("<.*?>")
        re_url = re.compile("https?://\S+|www\.\S+")
        re_user = re.compile("@\w+")
        wo_html_str = re_html.sub(r"", text)
        wo_url_str = re_url.sub("URL", wo_html_str)
        wo_user_str = re_user.sub("USER", wo_url_str)
        return wo_user_str
    # REPLACING SWEAR WORDS
    def replace_swear_words(self, text):
        text = profanity.censor(text, "-")
        return text
    # REMOVING PUNCTUATIONS & NUMBERS
    def remove_punctuation_number(self, text):
        text = text.str.replace('[{}]'.format(string.punctuation), "", regex=True)
        text = text.str.replace("\d+", "", regex=True)
        return text 
    # LEMMATISATION
    def lemmatisation(self, text):
        lemmatiser = WordNetLemmatizer()
        return lemmatiser.lemmatize(text)
    # TOKENISING
    def tokenise(self, text):
        tokenizer = TweetTokenizer()
        text = tokenizer.tokenize(text)
        return text
    # REPLACING EMOJIS TO TEXTS
    def demojise(self, text):
        text = [emoji.demojize(word)[1:-1] if emoji.demojize(word) != word else word for word in text]
        return text
    # REPLACING SWEAR WORDS TO ANGRY
    def replace_swear_word_to_angry(self, text):
        text = text.replace("----", "angry")
        return text
    # APPLYING ALL METHODS
    def clean(self, X):
        res = self.convert_lowercase(X)
        res = res.apply(self.remove_html_url)
        res = self.remove_punctuation_number(res)
        #res = res.apply(self.replace_swear_words) - we removed this step in the end due to better results without it 
        #res = res.apply(self.replace_swear_word_to_angry) - we removed this step in the end due to better results without it 
        res = res.apply(self.lemmatisation)
        res = res.apply(self.tokenise)
        res = res.apply(self.demojise)
        return res

In [15]:
#Applying the class defined above to clean and pre-process the data 
data_cleaning = DataCleaning()
X_train = data_cleaning.clean(X_train_raw) #applying to train dataset
X_test = data_cleaning.clean(X_test_raw) #applying to test dataset

# Save Pre-Processed Data

In [16]:
#Checking what type is our data 
type(X_train)

pandas.core.series.Series

In [17]:
#We convert it to pandas dataframe
X_train = X_train.to_frame()
X_test = X_test.to_frame()

In [18]:
#Just making sure what is the data format 
X_train.head()

,Tweet
0,"[USER, USER, shut, up, hashtags, are, cool, of..."
1,"[it, makes, me, so, fucking, irate, jesus, nob..."
2,"[lol, adam, the, bull, with, his, fake, outrage]"
3,"[USER, passed, away, early, this, morning, in,..."
4,"[USER, lol, wow, i, was, gonna, say, really, h..."


In [19]:
#We don't need to pre-process the labels 
y_train = y_train_raw
y_test = y_test_raw

In [20]:
#We convert the labels to dataframe
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [21]:
#Saving the pre-processed data
X_train.to_csv("/content/drive/My Drive/ML4NLP_Assignment5/X_train_2.csv")
X_test.to_csv("/content/drive/My Drive/ML4NLP_Assignment5/X_test_2.csv")
y_train.to_csv("/content/drive/My Drive/ML4NLP_Assignment5/y_train_2.csv")
y_test.to_csv("/content/drive/My Drive/ML4NLP_Assignment5/y_test_2.csv")

# Load the pre-processed data

In [22]:
#Loading the pre-processed data
X_train = pd.read_csv("/content/drive/My Drive/ML4NLP_Assignment5/X_train_2.csv")
X_test = pd.read_csv("/content/drive/My Drive/ML4NLP_Assignment5/X_test_2.csv")
y_train = pd.read_csv("/content/drive/My Drive/ML4NLP_Assignment5/y_train_2.csv")
y_test = pd.read_csv("/content/drive/My Drive/ML4NLP_Assignment5/y_test_2.csv")

In [23]:
X_train.drop(columns=["Unnamed: 0"])

,Tweet
0,"['USER', 'USER', 'shut', 'up', 'hashtags', 'ar..."
1,"['it', 'makes', 'me', 'so', 'fucking', 'irate'..."
2,"['lol', 'adam', 'the', 'bull', 'with', 'his', ..."
3,"['USER', 'passed', 'away', 'early', 'this', 'm..."
4,"['USER', 'lol', 'wow', 'i', 'was', 'gonna', 's..."
...,...
1696,"['got', 'a', 'tip', 'from', 'a', 'drunk', 'ube..."
1697,"['USER', 'USER', 'USER', 'USER', 'fucker', 'bl..."
1698,"['USER', 'i', 'look', 'rabid']"
1699,"['USER', 'im', 'not', 'surprised', 'i', 'would..."


In [24]:
#We are dropping the extra column which is empty
X_train = X_train.drop(columns=["Unnamed: 0"])
X_test = X_test.drop(columns=["Unnamed: 0"])
y_train = y_train.drop(columns=["Unnamed: 0"])
y_test = y_test.drop(columns=["Unnamed: 0"])

# Data formating 

Concatenate frames - We want to keep the tweets and labels in the same dataset object later 

In [25]:
frames = [X_train, y_train]

In [26]:
train_data = pd.concat(frames, axis=1) #Concatenating tweets and labels for the train dataset in one dataframe

In [27]:
frames_test = [X_test, y_test]

In [28]:
test_data = pd.concat(frames_test, axis=1) #Concatenating tweets and labels for the test dataset in one dataframe

In [29]:
train_data.head()

,Tweet,Intensity Score
0,"['USER', 'USER', 'shut', 'up', 'hashtags', 'ar...",0.562
1,"['it', 'makes', 'me', 'so', 'fucking', 'irate'...",0.750
2,"['lol', 'adam', 'the', 'bull', 'with', 'his', ...",0.417
3,"['USER', 'passed', 'away', 'early', 'this', 'm...",0.354
4,"['USER', 'lol', 'wow', 'i', 'was', 'gonna', 's...",0.438


In [30]:
test_data.head()

,Tweet,Intensity Score
0,"['USER', 'i', 'know', 'you', 'mean', 'well', '...",0.734
1,"['let', 'go', 'of', 'resentment', 'it', 'will'...",0.422
2,"['no', 'im', 'not', 'depressed', 'because', 'o...",0.663
3,"['amarnathterrorattack', 'muslims', 'are', 'ki...",0.703
4,"['prepare', 'to', 'suffer', 'the', 'sting', 'o...",0.719


In [31]:
#Renaming columns in our train dataframe
train_data.rename(columns = {'Tweet':'text'}, inplace = True)

In [32]:
train_data.rename(columns = {'Intensity Score':'label'}, inplace = True)

In [33]:
#Checking the end result
train_data.head()

,text,label
0,"['USER', 'USER', 'shut', 'up', 'hashtags', 'ar...",0.562
1,"['it', 'makes', 'me', 'so', 'fucking', 'irate'...",0.750
2,"['lol', 'adam', 'the', 'bull', 'with', 'his', ...",0.417
3,"['USER', 'passed', 'away', 'early', 'this', 'm...",0.354
4,"['USER', 'lol', 'wow', 'i', 'was', 'gonna', 's...",0.438


In [34]:
#Renaming columns in our test dataframe
test_data.rename(columns = {'Tweet':'text'}, inplace = True)
test_data.rename(columns = {'Intensity Score':'label'}, inplace = True)

In [35]:
#Checking the end result
test_data.head()

,text,label
0,"['USER', 'i', 'know', 'you', 'mean', 'well', '...",0.734
1,"['let', 'go', 'of', 'resentment', 'it', 'will'...",0.422
2,"['no', 'im', 'not', 'depressed', 'because', 'o...",0.663
3,"['amarnathterrorattack', 'muslims', 'are', 'ki...",0.703
4,"['prepare', 'to', 'suffer', 'the', 'sting', 'o...",0.719


Splitting train_data into train_data and val_data for hyperparametres tuning

In [36]:
#first shuffle the data randomly 
shuffled_train = train_data.sample(frac=1)

In [37]:
len(shuffled_train)

1701

In [38]:
# Splitting the train dataset into validation dataset and new train datatset
# approximately 15% of the initial train dataset will go to the validation dataset
train_data = shuffled_train[:1446]
val_data = shuffled_train[1446:]

In [39]:
#Importing more libraries and packages
import transformers
from datasets import Dataset,load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [40]:
#Getting the overview of the current training data format 
train_data.head()

,text,label
1677,"['a', 'USER', 'not', 'turning', 'up', 'why', '...",0.688
635,"['firsttweetever', 'sippin', 'hotchocolate', '...",0.542
1049,"['im', 'just', 'doing', 'what', 'u', 'should',...",0.312
459,"['USER', 'USER', 'what', 'cant', 'you', 'grasp...",0.750
422,"['USER', 'hi', 'monica', 'i', 'write', 'regula...",0.375


In [41]:
train_data.loc[738]

text     ['ellie', 'just', 'gave', 'me', 'loads', 'of',...
label                                                0.146
Name: 738, dtype: object

Convert train, val, and test datasets from Dataframe to Dataset Object

In [42]:
# Coverting dataframes to dataset objects 
train_dataset = Dataset.from_pandas(train_data, preserve_index=False)
test_dataset = Dataset.from_pandas(test_data, preserve_index=False)
val_dataset = Dataset.from_pandas(val_data, preserve_index=False)

In [43]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1446
})

In [44]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1002
})

In [45]:
val_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 255
})

In [46]:
val_dataset[2]

{'text': "['good', 'luck', 'to', 'all', 'furyhaney', 'players', 'playing', 'this', 'weekend', 'at', 'the', 'future', 'stars', 'showcase', 'in', 'frisco', 'tx', 'kgb', 'gtown', 'fury']",
 'label': 0.396}

In [47]:
type(val_dataset[2]['text'])

str

Above we can see that what should be a list, is represented as a string in our Datasets object. Hence, we fix it below:

In [48]:
import ast

def fix_dataset(example):
  example['text'] = ast.literal_eval(example['text'])
  return example

In [49]:
val_dataset_fixed = val_dataset.map(fix_dataset)

  0%|          | 0/255 [00:00<?, ?ex/s]

In [50]:
val_dataset_fixed[2]

{'text': ['good',
  'luck',
  'to',
  'all',
  'furyhaney',
  'players',
  'playing',
  'this',
  'weekend',
  'at',
  'the',
  'future',
  'stars',
  'showcase',
  'in',
  'frisco',
  'tx',
  'kgb',
  'gtown',
  'fury'],
 'label': 0.396}

In [51]:
train_dataset_fixed = train_dataset.map(fix_dataset)

  0%|          | 0/1446 [00:00<?, ?ex/s]

In [52]:
test_dataset_fixed = test_dataset.map(fix_dataset)

  0%|          | 0/1002 [00:00<?, ?ex/s]

In [53]:
train_dataset_fixed[2]['text']

['im',
 'just',
 'doing',
 'what',
 'u',
 'should',
 'b',
 'doing',
 'just',
 'minding',
 'my',
 'business',
 'and',
 'grinding',
 'relentless',
 'USER']

De-tokenize.
We already tokenized the dataset in the pre-processing step, but then we decided to use the HuggingFace Tokenizer instead. So we join the words again

In [54]:
#Function to join the words into one string per twit
def join_tweets(example):
  example['text'] = " ".join(example['text'])
  return example

In [55]:
#Applying this to the validation dataset
val_dataset_join = val_dataset_fixed.map(join_tweets)

  0%|          | 0/255 [00:00<?, ?ex/s]

In [56]:
val_dataset_join[2]

{'text': 'good luck to all furyhaney players playing this weekend at the future stars showcase in frisco tx kgb gtown fury',
 'label': 0.396}

In [57]:
#Applying this to the train and test datasets
train_dataset_join = train_dataset_fixed.map(join_tweets)
test_dataset_join = test_dataset_fixed.map(join_tweets)

  0%|          | 0/1446 [00:00<?, ?ex/s]

  0%|          | 0/1002 [00:00<?, ?ex/s]

In [58]:
train_dataset_join[2]

{'text': 'im just doing what u should b doing just minding my business and grinding relentless USER',
 'label': 0.312}

In [59]:
lenghts_train = []

for a in range(len(train_dataset_join)):
  txt = train_dataset_join[a]['text']
  lenghts_train.append(len(txt))

In [60]:
#Check what is the length of the longest string/tweet
max(lenghts_train)

268

Tokenize the pre-trained model

In [61]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [62]:
len(tokenizer)

30522

In [63]:
#we want to padd all tweets to the length of the longest one
max_length = 268

In [64]:
#Now we tokenize the tweets again
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset_join.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset_join.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset_join.map(tokenize_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [65]:
#Download the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

In [66]:
#Re-size the pre-trained model to the len of the tokenizer
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [67]:
from sklearn.metrics import mean_squared_error

In [68]:
import scipy.stats

In [69]:
#Define the function to compute the metrics 
from datasets import load_metric

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    r, p = scipy.stats.pearsonr(labels, predictions)
    return {"rmse": rmse, "pearsonr": r}

# First model

In [70]:
#Define the training args 
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
#Train the first model
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1701
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 321
  Number of trainable parameters = 66954241


Epoch,Training Loss,Validation Loss,Rmse
1,0.014200,0.023599,0.153618
2,0.011300,0.018607,0.136407
3,0.006300,0.018800,0.137114


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 

TrainOutput(global_step=321, training_loss=0.010591309389964072, metrics={'train_runtime': 77.4272, 'train_samples_per_second': 65.907, 'train_steps_per_second': 4.146, 'total_flos': 675969080177664.0, 'train_loss': 0.010591309389964072, 'epoch': 3.0})

# Second model

In [70]:
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error

In [71]:
import numpy as np

In [72]:
#Different version of the function to compute the metrics 
def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    rmse = mean_squared_error(labels, logits, squared=False)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)
    

    return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "smape": smape}

In [ ]:
#Training arguments for the second model
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics_for_regression
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
#Training the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1701
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 321
  Number of trainable parameters = 66954241


Epoch,Training Loss,Validation Loss,Mse,Rmse,Mae,R2,Smape
1,0.007700,0.021191,0.021191,0.145572,0.114679,0.409509,24.137257
2,0.006100,0.019650,0.019650,0.140178,0.112058,0.452464,23.653032
3,0.003900,0.018500,0.018500,0.136013,0.107576,0.484511,22.893147


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 

TrainOutput(global_step=321, training_loss=0.005878259274075707, metrics={'train_runtime': 77.9786, 'train_samples_per_second': 65.441, 'train_steps_per_second': 4.117, 'total_flos': 675969080177664.0, 'train_loss': 0.005878259274075707, 'epoch': 3.0})

In [ ]:
#Evaluate the model
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 16


{'eval_loss': 0.018499672412872314,
 'eval_mse': 0.018499670550227165,
 'eval_rmse': 0.13601349294185638,
 'eval_mae': 0.10757619887590408,
 'eval_r2': 0.484510977098937,
 'eval_smape': 22.893147299151696,
 'eval_runtime': 4.3911,
 'eval_samples_per_second': 228.191,
 'eval_steps_per_second': 14.347,
 'epoch': 3.0}

# Third MODEL

In [ ]:
#Arguments for the third model
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=64,
                                  per_device_eval_batch_size=20,
                                  weight_decay = 0.01, 
                                  learning_rate = 2e-5,
                                  num_train_epochs=10,
                                  save_total_limit = 10,
                                  save_strategy = 'epoch',
                                  load_best_model_at_end=True
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics_for_regression
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
#Training the third model
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1701
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 270
  Number of trainable parameters = 66954241


Epoch,Training Loss,Validation Loss,Mse,Rmse,Mae,R2,Smape
1,0.003100,0.019845,0.019845,0.140873,0.112157,0.447020,23.871443
2,0.003400,0.018579,0.018579,0.136306,0.107555,0.482293,22.882148
3,0.003000,0.018530,0.018530,0.136125,0.107245,0.483669,22.812531
4,0.002600,0.018952,0.018952,0.137665,0.109006,0.471917,23.212203
5,0.002500,0.018704,0.018704,0.136764,0.108045,0.478806,23.025921
6,0.002400,0.019109,0.019109,0.138237,0.109671,0.467523,23.328195
7,0.002400,0.018377,0.018377,0.135560,0.107272,0.487943,22.883645
8,0.002200,0.018475,0.018475,0.135922,0.107592,0.485203,22.961912
9,0.002000,0.018683,0.018683,0.136684,0.108142,0.479413,23.060216
10,0.002100,0.018602,0.018602,0.136388,0.107823,0.481668,23.000450


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 20
Saving model checkpoint to test_trainer/checkpoint-27
Configuration saved in test_trainer/checkpoint-27/config.json
Model weights saved in test_trainer/checkpoint-27/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 20
Saving model checkpoint to test_trainer/checkpoint-54
Configuration saved in test_trainer/checkpoint-54/config.json
Model weights save

TrainOutput(global_step=270, training_loss=0.0025765578641935632, metrics={'train_runtime': 254.7842, 'train_samples_per_second': 66.762, 'train_steps_per_second': 1.06, 'total_flos': 2253230267258880.0, 'train_loss': 0.0025765578641935632, 'epoch': 10.0})

In [ ]:
#Evaluate the third model
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 20


{'eval_loss': 0.018376503139734268,
 'eval_mse': 0.018376503139734268,
 'eval_rmse': 0.13555996119976044,
 'eval_mae': 0.107271708548069,
 'eval_r2': 0.4879430731052734,
 'eval_smape': 22.883644819735526,
 'eval_runtime': 4.1423,
 'eval_samples_per_second': 241.895,
 'eval_steps_per_second': 12.312,
 'epoch': 10.0}

# Fourth Model (trying FLAML)

In [73]:
from transformers import TrainingArguments, Trainer

In [74]:
MODEL_CHECKPOINT = "distilbert-base-uncased"

In [75]:
#Download the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [ ]:
#Define training arguments 
args = TrainingArguments(
    output_dir='output',
    do_eval=True,
)

In [ ]:
#Define the trainer 
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_for_regression,
)

In [ ]:
#Train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1701
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 639
  Number of trainable parameters = 66954241
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than

Step,Training Loss
500,0.018600


Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in output/checkpoint-500/tokenizer_config.json
Special tokens file saved in output/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=639, training_loss=0.016356775271873892, metrics={'train_runtime': 74.3096, 'train_samples_per_second': 68.672, 'train_steps_per_second': 8.599, 'total_flos': 675969080177664.0, 'train_loss': 0.016356775271873892, 'epoch': 3.0})

In [ ]:
#Evaluate the model
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 8


{'eval_loss': 0.019569557160139084,
 'eval_mse': 0.019569557160139084,
 'eval_rmse': 0.13989123702049255,
 'eval_mae': 0.11179779469966888,
 'eval_r2': 0.45469886176708085,
 'eval_smape': 23.760578452470057,
 'eval_runtime': 4.3975,
 'eval_samples_per_second': 227.854,
 'eval_steps_per_second': 28.652,
 'epoch': 3.0}

In [ ]:
#Install flaml
!pip install flaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Import the flaml
import flaml

In [ ]:
#Define the function for the ray tuning 

def train_distilbert(config: dict):

    

    train_dataset, eval_dataset = tokenized_train_dataset, tokenized_val_dataset #Pass the train and evaluation sets for tuning

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_CHECKPOINT, num_labels=1
    )

    def compute_metrics_for_regression(eval_pred): #Function to compute the metrics for evaluation 
        logits, labels = eval_pred
        labels = labels.reshape(-1, 1)

        mse = mean_squared_error(labels, logits)
        rmse = mean_squared_error(labels, logits, squared=False)
        mae = mean_absolute_error(labels, logits)
        r2 = r2_score(labels, logits)
        smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)

        return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "smape": smape}

    #Training args 
    training_args = TrainingArguments(
        output_dir='.',
        do_eval=False,
        disable_tqdm=True,
        logging_steps=20000,
        save_total_limit=0,
        **config,
    )
    #Trainer
    trainer = Trainer(
        model,
        training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_for_regression,
    )

    # train model
    trainer.train()

    # evaluate model
    eval_output = trainer.evaluate()

    # report the metric to optimize
    flaml.tune.report(
        r2=eval_output["eval_r2"],
    )

In [ ]:
#Define the search space for ray tuning 
max_num_epoch = 64
search_space = {
        # You can mix constants with search space objects.
        "num_train_epochs": flaml.tune.loguniform(1, max_num_epoch),
        "learning_rate": flaml.tune.loguniform(1e-6, 1e-4),
        "adam_epsilon": flaml.tune.loguniform(1e-9, 1e-7),
        "adam_beta1": flaml.tune.uniform(0.8, 0.99),
        "adam_beta2": flaml.tune.loguniform(98e-2, 9999e-4),
}

In [ ]:
# optimization objective
HP_METRIC, MODE = "r2", "max"

# resources
num_cpus = 0
num_gpus = 1

# constraints
num_samples = -1    # number of trials, -1 means unlimited
time_budget_s = 3600    # time budget in seconds

In [ ]:
!pip install pickle5

In [ ]:
import pickle5 as pickle

In [ ]:
#Ray tuning 
import time
import ray
start_time = time.time()
ray.shutdown()
ray.init(num_cpus=num_cpus, num_gpus=num_gpus)

print("Tuning started...")
analysis = flaml.tune.run(
    train_distilbert,
    search_alg=flaml.CFO(
        space=search_space,
        metric=HP_METRIC,
        mode=MODE,
        low_cost_partial_config={"num_train_epochs": 1}),
    # uncomment the following if scheduler = 'asha',
    # max_resource=max_num_epoch, min_resource=1,
    resources_per_trial={"gpu": num_gpus, "cpu": num_cpus},
    local_dir='logs/',
    num_samples=num_samples,
    time_budget_s=time_budget_s,
    use_ray=True,
)

ray.shutdown()

Tuning started...


(train_distilbert pid=7468) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
(train_distilbert pid=7468) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train_distilbert pid=7468) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train_distilbert pid=7468) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoin

(train_distilbert pid=7468) 
(train_distilbert pid=7468) 
(train_distilbert pid=7468) Training completed. Do not forget to share your model on huggingface.co/models =)
(train_distilbert pid=7468) 
(train_distilbert pid=7468) 
(train_distilbert pid=7468) The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
(train_distilbert pid=7468) ***** Running Evaluation *****
(train_distilbert pid=7468)   Num examples = 1002
(train_distilbert pid=7468)   Batch size = 8


(train_distilbert pid=7468) {'train_runtime': 23.43, 'train_samples_per_second': 72.599, 'train_steps_per_second': 9.091, 'train_loss': 0.06773512576107688, 'epoch': 1.0}


Trial train_distilbert_75886418 reported r2=0.04 with parameters={'num_train_epochs': 1.0, 'learning_rate': 1.5662610420278344e-06, 'adam_epsilon': 1.9702991167906198e-08, 'adam_beta1': 0.8689337534108345, 'adam_beta2': 0.9872898093714128}.
Trial train_distilbert_75886418 completed. Last result: r2=0.03501930286920463
(train_distilbert pid=7468) {'eval_loss': 0.03463085740804672, 'eval_mse': 0.03463085740804672, 'eval_rmse': 0.18609367311000824, 'eval_mae': 0.15341486036777496, 'eval_r2': 0.03501930286920463, 'eval_smape': 31.362470371756483, 'eval_runtime': 4.2255, 'eval_samples_per_second': 237.131, 'eval_steps_per_second': 29.819, 'epoch': 1.0}


...

Trial name,status,loc,adam_beta1,adam_beta2,adam_epsilon,learning_rate,num_train_epochs,iter,total time (s),r2
train_distilbert_75886418,TERMINATED,172.28.0.2:7468,0.868934,0.98729,1.9703e-08,1.56626e-06,1,1,30.3793,0.0350193
train_distilbert_797cc10e,TERMINATED,172.28.0.2:7558,0.94368,0.980268,3.41024e-09,3.2432e-05,1.44427,1,40.3436,0.420948
train_distilbert_8c1c4f3c,TERMINATED,172.28.0.2:7656,0.83925,0.989131,3.57728e-08,1.46966e-06,1,1,30.4707,0.00501224
train_distilbert_a81fc376,TERMINATED,172.28.0.2:7747,0.898618,0.985452,1.08521e-08,1.66921e-06,1.02723,1,31.2992,0.00896462
train_distilbert_be2eb96a,TERMINATED,172.28.0.2:7839,0.892053,0.987263,1.7536e-08,1.41402e-06,1,1,30.5941,0.019258
train_distilbert_d4d3bcce,TERMINATED,172.28.0.2:7932,0.845815,0.987317,2.21377e-08,1.73489e-06,2.15443,1,56.6846,0.104871
train_distilbert_eae45e74,TERMINATED,172.28.0.2:8035,0.856788,0.988564,3.92079e-08,2.81634e-06,1,1,30.5621,0.0509921
train_distilbert_110ae6f4,TERMINATED,172.28.0.2:8128,0.857961,0.986044,1.11248e-08,1e-06,2.72935,1,71.3863,0.047188
train_distilbert_27172e26,TERMINATED,172.28.0.2:8238,0.806751,0.987522,2.67955e-08,1.95709e-06,1.59538,1,44.2964,0.143462
train_distilbert_55961c9e,TERMINATED,172.28.0.2:8336,0.884879,0.987112,1.82896e-08,1.53792e-06,2.90939,1,75.5793,0.181381


(train_distilbert pid=10665) Saving model checkpoint to ./checkpoint-3500
(train_distilbert pid=10665) Configuration saved in ./checkpoint-3500/config.json
(train_distilbert pid=10665) Model weights saved in ./checkpoint-3500/pytorch_model.bin
(train_distilbert pid=10665) tokenizer config file saved in ./checkpoint-3500/tokenizer_config.json
(train_distilbert pid=10665) Special tokens file saved in ./checkpoint-3500/special_tokens_map.json
(train_distilbert pid=10665) Saving model checkpoint to ./checkpoint-4000
(train_distilbert pid=10665) Configuration saved in ./checkpoint-4000/config.json
(train_distilbert pid=10665) Model weights saved in ./checkpoint-4000/pytorch_model.bin
(train_distilbert pid=10665) tokenizer config file saved in ./checkpoint-4000/tokenizer_config.json
(train_distilbert pid=10665) Special tokens file saved in ./checkpoint-4000/special_tokens_map.json
(train_distilbert pid=10665) 
(train_distilbert pid=10665) 
(train_distilbert pid=10665) Training completed. Do 

(train_distilbert pid=10665) {'train_runtime': 474.7559, 'train_samples_per_second': 72.816, 'train_steps_per_second': 9.118, 'train_loss': 0.010420797713278182, 'epoch': 20.32}


2022-12-01 23:16:03,226	INFO tune.py:748 -- Total run time: 3722.07 seconds (3605.67 seconds for the tuning loop).


(train_distilbert pid=10665) {'eval_loss': 0.020890725776553154, 'eval_mse': 0.020890725776553154, 'eval_rmse': 0.14453624188899994, 'eval_mae': 0.11490651220083237, 'eval_r2': 0.4178848508485151, 'eval_smape': 24.54094545284431, 'eval_runtime': 4.2376, 'eval_samples_per_second': 236.453, 'eval_steps_per_second': 29.734, 'epoch': 20.32}


In [ ]:
#Get the results of hyperparameters optimization 
best_trial = analysis.get_best_trial(HP_METRIC, MODE, "all")
metric = best_trial.metric_analysis[HP_METRIC][MODE]
print(f"n_trials={len(analysis.trials)}")
print(f"time={time.time()-start_time}")
print(f"Best model eval {HP_METRIC}: {metric:.4f}")
print(f"Best model parameters: {best_trial.config}")

n_trials=26
time=3748.902177810669
Best model eval r2: 0.4615
Best model parameters: {'num_train_epochs': 14.90289944900019, 'learning_rate': 4.278615732972273e-06, 'adam_epsilon': 6.900956598306757e-08, 'adam_beta1': 0.841736350312499, 'adam_beta2': 0.9938714514369775}


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some we

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [ ]:
#Now use the best hyperparameters found to train the model
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_steps=20000,
                                  per_device_train_batch_size=64,
                                  per_device_eval_batch_size=20,
                                  learning_rate = 4.278615732972273e-06,
                                  adam_epsilon = 6.900956598306757e-08,
                                  adam_beta1 = 0.841736350312499,
                                  adam_beta2 = 0.9938714514369775,
                                  num_train_epochs=15,
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics_for_regression
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
#Train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1701
  Num Epochs = 15
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 405
  Number of trainable parameters = 66954241


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=405, training_loss=0.010689811942018108, metrics={'train_runtime': 287.6483, 'train_samples_per_second': 88.702, 'train_steps_per_second': 1.408, 'total_flos': 3379845400888320.0, 'train_loss': 0.010689811942018108, 'epoch': 15.0})

In [ ]:
#Evaluate the model
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 20


{'eval_loss': 0.02048257365822792,
 'eval_mse': 0.02048257552087307,
 'eval_rmse': 0.14311735332012177,
 'eval_mae': 0.11232398450374603,
 'eval_r2': 0.4292578633151708,
 'eval_smape': 23.861486012350298,
 'eval_runtime': 4.0928,
 'eval_samples_per_second': 244.819,
 'eval_steps_per_second': 12.461,
 'epoch': 15.0}

# Fifth model

In [ ]:
#Trying with different training arguments 
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_steps=20000,
                                  #per_device_train_batch_size=64,
                                  #per_device_eval_batch_size=20,
                                  learning_rate = 4.278615732972273e-06,
                                  adam_epsilon = 6.900956598306757e-08,
                                  adam_beta1 = 0.841736350312499,
                                  adam_beta2 = 0.9938714514369775,
                                  num_train_epochs=50,
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics_for_regression
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
#Train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1701
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10650
  Number of trainable parameters = 66954241


Step,Training Loss


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

TrainOutput(global_step=10650, training_loss=0.0033621011653416594, metrics={'train_runtime': 1220.3216, 'train_samples_per_second': 69.695, 'train_steps_per_second': 8.727, 'total_flos': 1.12661513362944e+16, 'train_loss': 0.0033621011653416594, 'epoch': 50.0})

In [ ]:
#Evaluate the model
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 8


{'eval_loss': 0.020279884338378906,
 'eval_mse': 0.020279884338378906,
 'eval_rmse': 0.14240746200084686,
 'eval_mae': 0.11204035580158234,
 'eval_r2': 0.4349057738233796,
 'eval_smape': 23.816269804141715,
 'eval_runtime': 4.446,
 'eval_samples_per_second': 225.371,
 'eval_steps_per_second': 28.34,
 'epoch': 50.0}

# New ray search and Sixth model

In [73]:
from transformers import TrainingArguments, Trainer

In [74]:
MODEL_CHECKPOINT = "distilbert-base-uncased"

In [34]:
#Download the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=1)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [35]:
#Re-size to the length of the tokenizer 
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [64]:
#Traing args
args = TrainingArguments(
    output_dir='output',
    do_eval=True,
)

In [65]:
#Define the trainer 
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_for_regression,
)

In [ ]:
#Train the model
trainer.train()

In [66]:
#Evaluate the model
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 8
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.4004843831062317,
 'eval_mse': 0.4004843831062317,
 'eval_rmse': 0.6328383684158325,
 'eval_mae': 0.6030510663986206,
 'eval_r2': -10.159403175849627,
 'eval_smape': 199.99999999999997,
 'eval_runtime': 7.6485,
 'eval_samples_per_second': 131.005,
 'eval_steps_per_second': 16.474}

In [36]:
!pip install flaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import flaml

In [69]:
def train_distilbert(config: dict): #Preparing the function for another tuning 

    

    train_dataset, eval_dataset = tokenized_train_dataset, tokenized_val_dataset

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_CHECKPOINT, num_labels=1
    )
    model.resize_token_embeddings(len(tokenizer))

    

    def compute_metrics_for_regression(eval_pred):
        logits, labels = eval_pred
        labels = labels.reshape(-1, 1)

        mse = mean_squared_error(labels, logits)
        rmse = mean_squared_error(labels, logits, squared=False)
        mae = mean_absolute_error(labels, logits)
        r2 = r2_score(labels, logits)
        smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)

        return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "smape": smape}


    training_args = TrainingArguments(
        output_dir='.',
        do_eval=False,
        disable_tqdm=True,
        logging_steps=20000,
        save_total_limit=0,
        **config,
    )

    trainer = Trainer(
        model,
        training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_for_regression,
    )

    # train model
    trainer.train()

    # evaluate model
    eval_output = trainer.evaluate()

    # report the metric to optimize
    flaml.tune.report(
        r2=eval_output["eval_r2"],
    )

In [70]:
############ Version 2 ########################
#Different search space for hyperparameters tuning

max_num_epoch = 64
search_space = {
        # You can mix constants with search space objects.
        "num_train_epochs": flaml.tune.loguniform(1, max_num_epoch),
        "learning_rate": flaml.tune.loguniform(1e-6, 1e-4),
        "adam_epsilon": flaml.tune.loguniform(1e-9, 1e-7),
        "adam_beta1": flaml.tune.uniform(0.8, 0.99),
        "adam_beta2": flaml.tune.loguniform(98e-2, 9999e-4),
        "weight_decay": flaml.tune.uniform(0.0, 0.3),
        
}

In [71]:
# optimization objective
HP_METRIC, MODE = "r2", "max"

# resources
num_cpus = 0
num_gpus = 1

# constraints
num_samples = -1    # number of trials, -1 means unlimited
time_budget_s = 3600    # time budget in seconds

In [38]:
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 15.8 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=236289 sha256=2f125c921bff129c58703213e7e09403daec129c23a997f1f3c2bf7eb4eb8261
  Stored in directory: /root/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5


In [39]:
import pickle5 as pickle

In [74]:
#Another ray hyperparameters tuning
import time
import ray
start_time = time.time()
ray.shutdown()
ray.init(num_cpus=num_cpus, num_gpus=num_gpus)

print("Tuning started...")
analysis = flaml.tune.run(
    train_distilbert,
    search_alg=flaml.CFO(
        space=search_space,
        metric=HP_METRIC,
        mode=MODE,
        low_cost_partial_config={"num_train_epochs": 1}),
    # uncomment the following if scheduler = 'asha',
    # max_resource=max_num_epoch, min_resource=1,
    resources_per_trial={"gpu": num_gpus, "cpu": num_cpus},
    local_dir='logs/',
    num_samples=num_samples,
    time_budget_s=time_budget_s,
    use_ray=True,
)

ray.shutdown()

2022-12-02 04:30:27,850	WARNING function_runner.py:603 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Tuning started...


(train_distilbert pid=1110) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
(train_distilbert pid=1110) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train_distilbert pid=1110) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train_distilbert pid=1110) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoin

(train_distilbert pid=1110) {'train_runtime': 25.0235, 'train_samples_per_second': 67.976, 'train_steps_per_second': 8.512, 'train_loss': 0.04155403907310235, 'epoch': 1.0}


(train_distilbert pid=1110) 
(train_distilbert pid=1110) 
(train_distilbert pid=1110) Training completed. Do not forget to share your model on huggingface.co/models =)
(train_distilbert pid=1110) 
(train_distilbert pid=1110) 
(train_distilbert pid=1110) The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
(train_distilbert pid=1110) ***** Running Evaluation *****
(train_distilbert pid=1110)   Num examples = 1002
(train_distilbert pid=1110)   Batch size = 8


...

Trial name,status,loc,adam_beta1,adam_beta2,adam_epsilon,learning_rate,num_train_epochs,weight_decay,iter,total time (s),r2
train_distilbert_0c5ec8d0,TERMINATED,172.28.0.12:1110,0.868934,0.98729,1.9703e-08,1.56626e-06,1,0.226646,1,32.8451,0.0371399
train_distilbert_10a85884,TERMINATED,172.28.0.12:1205,0.802586,0.995234,3.25403e-08,3.41024e-06,1.44427,0.0299459,1,41.8021,0.187267
train_distilbert_24aefc66,TERMINATED,172.28.0.12:1304,0.845374,0.988751,3.16312e-08,1.48909e-06,1,0.175998,1,31.6102,0.0108226
train_distilbert_41e4cf86,TERMINATED,172.28.0.12:1399,0.892494,0.985831,1.22729e-08,1.64743e-06,1.02155,0.277294,1,32.2443,0.0247796
train_distilbert_59242c46,TERMINATED,172.28.0.12:1493,0.868625,0.985727,3.84131e-08,1.36325e-06,1,0.279604,1,31.6326,0.0568407
train_distilbert_706258e2,TERMINATED,172.28.0.12:1587,0.869242,0.988855,1.01061e-08,1.7995e-06,1.1163,0.173688,1,34.4753,0.0423844
train_distilbert_87a3b280,TERMINATED,172.28.0.12:1683,0.857986,0.986129,5.72013e-08,1e-06,2.30995,0.289247,1,62.0988,0.0662426
train_distilbert_a4310376,TERMINATED,172.28.0.12:1791,0.879265,0.985325,2.5796e-08,2.02689e-06,1,0.269961,1,31.7421,0.045148
train_distilbert_c9cc61a2,TERMINATED,172.28.0.12:1887,0.841587,0.985338,3.73367e-08,1.02605e-06,1.45422,0.237318,1,42.1019,0.0161436
train_distilbert_e10cee68,TERMINATED,172.28.0.12:1984,0.874385,0.986921,8.76346e-08,1e-06,3.66923,0.3,1,95.0856,0.0706021


(train_distilbert pid=4416) Saving model checkpoint to ./checkpoint-1000
(train_distilbert pid=4416) Configuration saved in ./checkpoint-1000/config.json
(train_distilbert pid=4416) Model weights saved in ./checkpoint-1000/pytorch_model.bin
(train_distilbert pid=4416) tokenizer config file saved in ./checkpoint-1000/tokenizer_config.json
(train_distilbert pid=4416) Special tokens file saved in ./checkpoint-1000/special_tokens_map.json


(train_distilbert pid=4416) {'train_runtime': 166.0707, 'train_samples_per_second': 71.844, 'train_steps_per_second': 9.002, 'train_loss': 0.011085732246322377, 'epoch': 7.02}


(train_distilbert pid=4416) 
(train_distilbert pid=4416) 
(train_distilbert pid=4416) Training completed. Do not forget to share your model on huggingface.co/models =)
(train_distilbert pid=4416) 
(train_distilbert pid=4416) 
(train_distilbert pid=4416) The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
(train_distilbert pid=4416) ***** Running Evaluation *****
(train_distilbert pid=4416)   Num examples = 1002
(train_distilbert pid=4416)   Batch size = 8
2022-12-02 05:31:46,869	INFO tune.py:747 -- Total run time: 3679.02 seconds (3605.12 seconds for the tuning loop).


(train_distilbert pid=4416) {'eval_loss': 0.019225232303142548, 'eval_mse': 0.019225232303142548, 'eval_rmse': 0.13865508139133453, 'eval_mae': 0.11000686138868332, 'eval_r2': 0.4642933742508213, 'eval_smape': 23.33022626621756, 'eval_runtime': 4.2487, 'eval_samples_per_second': 235.839, 'eval_steps_per_second': 29.656, 'epoch': 7.02}


In [43]:
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_steps=20000,
                                  #per_device_train_batch_size=64,
                                  #per_device_eval_batch_size=20,
                                  learning_rate = 1.857295378811325e-05,
                                  adam_epsilon = 2.4916439803066257e-08,
                                  adam_beta1 = 0.8109062777371857,
                                  adam_beta2 = 0.9870507093459101,
                                  weight_decay = 0.2851666696787331,
                                  num_train_epochs=50,
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics_for_regression
)

In [44]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1701
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10650
  Number of trainable parameters = 66954241


Step,Training Loss


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

TrainOutput(global_step=10650, training_loss=0.0027950387157744646, metrics={'train_runtime': 1224.5335, 'train_samples_per_second': 69.455, 'train_steps_per_second': 8.697, 'total_flos': 1.12661513362944e+16, 'train_loss': 0.0027950387157744646, 'epoch': 50.0})

In [45]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 8


{'eval_loss': 0.018708493560552597,
 'eval_mse': 0.018708493560552597,
 'eval_rmse': 0.13677899539470673,
 'eval_mae': 0.10913343727588654,
 'eval_r2': 0.47869219056724877,
 'eval_smape': 23.115403957709578,
 'eval_runtime': 4.4244,
 'eval_samples_per_second': 226.471,
 'eval_steps_per_second': 28.478,
 'epoch': 50.0}

# Another Ray search and the Seventh Model

In [71]:
#New function to evaluate, now it includes the Person r 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    r, p = scipy.stats.pearsonr(labels, predictions)
    r = r.item()
    return {"rmse": rmse, "pearsonr": r}

In [75]:
#Download the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=1)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some we

In [102]:
#Re-size to the length of the tokenizer 
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [103]:
#Arguments for training 
args = TrainingArguments(
    output_dir='output',
    do_eval=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [104]:
#Define the trainer 
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [105]:
#Train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1446
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 543
  Number of trainable parameters = 66954241


Step,Training Loss
500,0.014800


Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in output/checkpoint-500/tokenizer_config.json
Special tokens file saved in output/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=543, training_loss=0.014244305549625094, metrics={'train_runtime': 216.7375, 'train_samples_per_second': 20.015, 'train_steps_per_second': 2.505, 'total_flos': 574633327417344.0, 'train_loss': 0.014244305549625094, 'epoch': 3.0})

In [106]:
#Evaluate the model
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 255
  Batch size = 8


{'eval_loss': 0.01071194838732481,
 'eval_rmse': 0.10349854826927185,
 'eval_pearsonr': 0.78164069617914,
 'eval_runtime': 4.5533,
 'eval_samples_per_second': 56.003,
 'eval_steps_per_second': 7.028,
 'epoch': 3.0}

In [107]:
!pip install flaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [108]:
import flaml

In [109]:
MODEL_CHECKPOINT = "distilbert-base-uncased"

In [110]:
def train_distilbert(config: dict):

    

    train_dataset, eval_dataset = tokenized_train_dataset, tokenized_val_dataset

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_CHECKPOINT, num_labels=1
    )

    model.resize_token_embeddings(len(tokenizer))

    def compute_metrics(eval_pred):

      predictions, labels = eval_pred
      rmse = mean_squared_error(labels, predictions, squared=False)
      r, p = scipy.stats.pearsonr(labels, predictions)
      r = r.item()
      return {"rmse": rmse, "pearsonr": r}


    training_args = TrainingArguments(
        output_dir='.',
        do_eval=False,
        disable_tqdm=True,
        logging_steps=20000,
        save_total_limit=0,
        **config,
    )

    trainer = Trainer(
        model,
        training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    # train model
    trainer.train()

    # evaluate model
    eval_output = trainer.evaluate()

    # report the metric to optimize
    flaml.tune.report(
        pearsonr=eval_output["eval_pearsonr"],
    )

In [111]:
############ Version 2 ########################
#New search space for the hyperparameters to optimize 

max_num_epoch = 64
search_space = {
        # You can mix constants with search space objects.
        "num_train_epochs": flaml.tune.loguniform(1, max_num_epoch),
        "learning_rate": flaml.tune.loguniform(1e-6, 1e-4),
        "adam_epsilon": flaml.tune.loguniform(1e-9, 1e-7),
        "adam_beta1": flaml.tune.uniform(0.8, 0.99),
        "adam_beta2": flaml.tune.loguniform(98e-2, 9999e-4),
        "weight_decay": flaml.tune.uniform(0.0, 0.3),
        
}

In [112]:
# optimization objective
HP_METRIC, MODE = "pearsonr", "max"

# resources
num_cpus = 0
num_gpus = 1

# constraints
num_samples = -1    # number of trials, -1 means unlimited
time_budget_s = 3600    # time budget in seconds

In [76]:
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [77]:
import pickle5 as pickle

In [115]:
#Another ray tuning 
import time
import ray
start_time = time.time()
ray.shutdown()
ray.init(num_cpus=num_cpus, num_gpus=num_gpus)

print("Tuning started...")
analysis = flaml.tune.run(
    train_distilbert,
    search_alg=flaml.CFO(
        space=search_space,
        metric=HP_METRIC,
        mode=MODE,
        low_cost_partial_config={"num_train_epochs": 1}),
    # uncomment the following if scheduler = 'asha',
    # max_resource=max_num_epoch, min_resource=1,
    resources_per_trial={"gpu": num_gpus, "cpu": num_cpus},
    local_dir='logs/',
    num_samples=num_samples,
    time_budget_s=time_budget_s,
    use_ray=True,
)

ray.shutdown()

2022-12-04 02:36:33,735	WARNING function_runner.py:603 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Tuning started...


(train_distilbert pid=2464) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
(train_distilbert pid=2464) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train_distilbert pid=2464) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train_distilbert pid=2464) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoin

(train_distilbert pid=2464) {'train_runtime': 74.6131, 'train_samples_per_second': 19.38, 'train_steps_per_second': 2.426, 'train_loss': 0.08657695707036646, 'epoch': 1.0}


(train_distilbert pid=2464) 
(train_distilbert pid=2464) 
(train_distilbert pid=2464) Training completed. Do not forget to share your model on huggingface.co/models =)
(train_distilbert pid=2464) 
(train_distilbert pid=2464) 
(train_distilbert pid=2464) The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
(train_distilbert pid=2464) ***** Running Evaluation *****
(train_distilbert pid=2464)   Num examples = 255
(train_distilbert pid=2464)   Batch size = 8


...

Trial name,status,loc,adam_beta1,adam_beta2,adam_epsilon,learning_rate,num_train_epochs,weight_decay,iter,total time (s),pearsonr
train_distilbert_77cd135a,TERMINATED,172.28.0.12:2464,0.868934,0.98729,1.9703e-08,1.56626e-06,1,0.226646,1,82.8108,0.281236
train_distilbert_7c710ec0,TERMINATED,172.28.0.12:2578,0.802586,0.995234,3.25403e-08,3.41024e-06,1.44427,0.0299459,1,112.5,0.534588
train_distilbert_aec07e42,TERMINATED,172.28.0.12:2702,0.845374,0.988751,3.16312e-08,1.48909e-06,1,0.175998,1,80.4575,0.323514
train_distilbert_f648e20e,TERMINATED,172.28.0.12:2814,0.892494,0.985831,1.22729e-08,1.64743e-06,1.02155,0.277294,1,81.9792,0.298422
train_distilbert_2a2cfccc,TERMINATED,172.28.0.12:2924,0.845066,0.987186,6.16685e-08,1.29608e-06,1,0.228957,1,80.3606,0.470215
train_distilbert_5f35023e,TERMINATED,172.28.0.12:3036,0.845682,0.990318,1.62244e-08,1.71084e-06,1.1163,0.12304,1,90.1372,0.338453
train_distilbert_93139d68,TERMINATED,172.28.0.12:3144,0.834426,0.987588,9.18311e-08,1e-06,2.30995,0.2386,1,174.53,0.4316
train_distilbert_d064b9d6,TERMINATED,172.28.0.12:3263,0.855705,0.986783,4.1413e-08,1.92702e-06,1,0.219314,1,81.1941,0.352282
train_distilbert_3cde6616,TERMINATED,172.28.0.12:3357,0.828666,0.986393,4.02525e-08,1.32984e-06,1,0.177027,1,80.4917,0.138756
train_distilbert_6dd4014a,TERMINATED,172.28.0.12:3450,0.861465,0.987978,9.44785e-08,1.26318e-06,1.58844,0.280886,1,122.462,0.422607


(train_distilbert pid=5828) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
(train_distilbert pid=5828) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train_distilbert pid=5828) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train_distilbert pid=5828) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoin

(train_distilbert pid=5828) {'train_runtime': 73.5021, 'train_samples_per_second': 19.673, 'train_steps_per_second': 2.463, 'train_loss': 0.022954408635092044, 'epoch': 1.0}


2022-12-04 03:38:02,516	INFO tune.py:747 -- Total run time: 3688.79 seconds (3606.01 seconds for the tuning loop).


(train_distilbert pid=5828) {'eval_loss': 0.011115641333162785, 'eval_rmse': 0.10543074458837509, 'eval_pearsonr': 0.7644832697747332, 'eval_runtime': 4.5028, 'eval_samples_per_second': 56.631, 'eval_steps_per_second': 7.107, 'epoch': 1.0}


In [78]:
#Apply the best hyperparameters 
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_steps=20000,
                                  #per_device_train_batch_size=64,
                                  #per_device_eval_batch_size=20,
                                  learning_rate = 7.191707204279362e-05,
                                  adam_epsilon = 1e-07,
                                  adam_beta1 = 0.8148103444617636,
                                  adam_beta2 = 0.986557241529914,
                                  weight_decay = 0.23585877049061496,
                                  num_train_epochs=3,
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [79]:
#Train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1446
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 543
  Number of trainable parameters = 66954241


Step,Training Loss


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=543, training_loss=0.0134967598467242, metrics={'train_runtime': 212.8491, 'train_samples_per_second': 20.381, 'train_steps_per_second': 2.551, 'total_flos': 574633327417344.0, 'train_loss': 0.0134967598467242, 'epoch': 3.0})

In [80]:
#Evaluate the model
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 255
  Batch size = 8


{'eval_loss': 0.01217750832438469,
 'eval_rmse': 0.11035174876451492,
 'eval_pearsonr': 0.7761107852886949,
 'eval_runtime': 4.1049,
 'eval_samples_per_second': 62.12,
 'eval_steps_per_second': 7.796,
 'epoch': 3.0}

# Testing the Final Model on the test dataset

In [81]:
trainer.save_model("distilbert_classification")

Saving model checkpoint to distilbert_classification
Configuration saved in distilbert_classification/config.json
Model weights saved in distilbert_classification/pytorch_model.bin


In [82]:
predictions = trainer.predict(tokenized_test_dataset)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1002
  Batch size = 8


In [83]:
type(predictions)

transformers.trainer_utils.PredictionOutput

In [84]:
predictions[0]

array([[0.53839153],
       [0.45939144],
       [0.45762992],
       ...,
       [0.46221837],
       [0.7500934 ],
       [0.52689624]], dtype=float32)

In [85]:
predictions[1]

array([0.734, 0.422, 0.663, ..., 0.424, 0.597, 0.547], dtype=float32)

In [86]:
predictions[2]

{'test_loss': 0.015755170956254005,
 'test_rmse': 0.12551960349082947,
 'test_pearsonr': 0.7619375508507787,
 'test_runtime': 16.4978,
 'test_samples_per_second': 60.735,
 'test_steps_per_second': 7.637}

In [87]:
trainer.eval_dataset=tokenized_test_dataset

In [88]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1002
  Batch size = 8


{'eval_loss': 0.015755170956254005,
 'eval_rmse': 0.12551960349082947,
 'eval_pearsonr': 0.7619375508507787,
 'eval_runtime': 16.2302,
 'eval_samples_per_second': 61.737,
 'eval_steps_per_second': 7.763,
 'epoch': 3.0}